In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd

# Load the data
df = pd.read_csv(r"C:\Users\Bizhan\Downloads\Electric_Vehicle_Population_Data.csv")

# Select the columns that will be used for the prediction
features = ['County', 'City', 'Model Year', 'Make', 'Model', 'Electric Vehicle Type', 'Electric Range', 'Base MSRP', 'Electric Utility']
target = 'Make'

# Preprocess the data
for column in features:
    if df[column].dtype == 'object':
        # If the column is categorical, convert it to numerical values
        le = LabelEncoder()
        df[column] = le.fit_transform(df[column])

# Split the data into features (X) and target (y)
X = df[features]
y = df[target]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Load the pretrained model and tokenizer
model_name = 'tabnet'
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Convert the training data to PyTorch tensors
train_encodings = tokenizer(X_train.to_dict('records'), truncation=True, padding=True, max_length=512)
train_labels = torch.tensor(y_train.values)

# Fine-tune the model on the training data
model.train()
for epoch in range(10):  # Number of training epochs
    for i in range(len(train_encodings['input_ids'])):
        inputs = {key: torch.tensor(val[i]) for key, val in train_encodings.items()}
        labels = train_labels[i]
        outputs = model(**inputs, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

# Evaluate the model on the testing data
# Put the model in evaluation mode
model.eval()

# Convert the testing data to PyTorch tensors
test_encodings = tokenizer(X_test.to_dict('records'), truncation=True, padding=True, max_length=512)
test_labels = torch.tensor(y_test.values)

# Initialize a list to store the model's predictions
predictions = []

# Loop over the testing data
for i in range(len(test_encodings['input_ids'])):
    inputs = {key: torch.tensor(val[i]) for key, val in test_encodings.items()}

    # Feed the inputs to the model
    with torch.no_grad():
        outputs = model(**inputs)

    # Get the model's prediction
    _, predicted = torch.max(outputs.logits, 1)

    # Add the prediction to the list
    predictions.append(predicted.item())


DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


OSError: tabnet is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`